In [7]:
import netCDF4 as nc
import numpy as np
import datetime

# Open the netCDF
ncfile = nc.Dataset('/home/mliaqat/Wflwo/nc file spatial mean/2019.nc', 'r')

# Read in the evapotranspiration and interception data variables and their dimensions
et_data = ncfile.variables['actevap'][:]
interception_data = ncfile.variables['interception'][:]
# Replace values of with NaNs. In this case # -3.402823e+38
et_data = np.where(et_data == -3.402823e+38, np.nan, et_data)
interception_data = np.where(interception_data == -3.402823e+38, np.nan, interception_data)
time = ncfile.variables['time'][:]
lat = ncfile.variables['lat'][:]
lon = ncfile.variables['lon'][:]

# Convert values to datetime format
base_date = datetime.datetime(1900, 1, 2)
time_units = ncfile.variables['time'].units
time_cal = ncfile.variables['time'].calendar
time_dates = [base_date + datetime.timedelta(days=t) for t in time]

# Compute  8-day composites from daily data
num_composites = int(np.floor(len(time) / 8))

# Initialize arrays to keep the composite data and dates
composite_et_data = np.zeros((num_composites, len(lat), len(lon)))
composite_interception_data = np.zeros((num_composites, len(lat), len(lon)))
composite_data = np.zeros((num_composites, len(lat), len(lon)))
composite_dates = []

# Apply Loop through each 8-day composite and calculate the average of evapotranspiration and interception
for i in range(num_composites):
    start_index = i * 8
    end_index = start_index + 8
    composite_et_data[i,:,:] = np.sum(et_data[start_index:end_index,:,:], axis=0)
    composite_interception_data[i,:,:] = np.sum(interception_data[start_index:end_index,:,:], axis=0)
    composite_data[i,:,:] = composite_et_data[i,:,:] + composite_interception_data[i,:,:]
    composite_dates.append(time_dates[end_index-1])

# Create a new netCDF file to write the composite data
composite_ncfile = nc.Dataset('/home/mliaqat/Wflwo/nc file spatial mean/composite_ETact_2019.nc', 'w', format='NETCDF4')

# Create dimensions for the new file
composite_ncfile.createDimension('time', num_composites)
composite_ncfile.createDimension('lat', len(lat))
composite_ncfile.createDimension('lon', len(lon))

# Create variables for the new file
composite_time = composite_ncfile.createVariable('time', np.float64, ('time',))
composite_lat = composite_ncfile.createVariable('lat', np.float32, ('lat',))
composite_lon = composite_ncfile.createVariable('lon', np.float32, ('lon',))
composite_et = composite_ncfile.createVariable('et', np.float32, ('time','lat','lon',))

# Add attributes to the variables
composite_time.units = time_units
composite_time.calendar = time_cal

# Write the data to the new file
composite_time[:] = nc.date2num(composite_dates, units=time_units, calendar=time_cal)
composite_lat[:] = lat
composite_lon[:] = lon
composite_et[:] = composite_data

